#Cross-encoder

In [1]:
!pip install transformers datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset

datasets = load_dataset("PhilipMay/stsb_multi_mt", "en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/470k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})

In [6]:
datasets['train'][0]

{'sentence1': 'A plane is taking off.',
 'sentence2': 'An air plane is taking off.',
 'similarity_score': 5.0}

In [7]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

In [9]:
# Tokenize the sentences
train_inputs = tokenizer(
    datasets["train"]["sentence1"],
    datasets["train"]["sentence2"],
    padding='longest',    # Add padding to ensure consistent length (optional)
    return_tensors='pt'      # Return PyTorch tensors
)

In [10]:
train_inputs['input_ids'][0]

tensor([ 101, 1037, 4946, 2003, 2635, 2125, 1012,  102, 2019, 2250, 4946, 2003,
        2635, 2125, 1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])

In [11]:
# Tokenize the sentences
valid_inputs = tokenizer(
    datasets["dev"]["sentence1"],
    datasets["dev"]["sentence2"],
    padding='longest',    # Add padding to ensure consistent length (optional)
    return_tensors='pt'      # Return PyTorch tensors
)

In [12]:
# Tokenize the sentences
test_inputs = tokenizer(
    datasets["test"]["sentence1"],
    datasets["test"]["sentence2"],
    padding='longest',    # Add padding to ensure consistent length (optional)
    return_tensors='pt'      # Return PyTorch tensors
)

In [13]:
def normalize_tensor_range(tensor, new_min=-0, new_max=1):

    min_val = tensor.min()
    max_val = tensor.max()
    print(min_val, max_val)

    # Scale to [0, 1]
    tensor_scaled = (tensor - min_val) / (max_val - min_val)

    # Scale to [new_min, new_max]
    tensor_normalized = tensor_scaled * (new_max - new_min) + new_min

    return tensor_normalized

# Example tensors
labels = torch.tensor(datasets['train']['similarity_score'])
valid_labels = torch.tensor(datasets['dev']['similarity_score'])
test_labels = torch.tensor(datasets['test']['similarity_score'])


# Normalize tensors to the range [-1, 1]
normalized_labels = normalize_tensor_range(labels)
valid_normalized_labels = normalize_tensor_range(valid_labels)
test_normalized_labels = normalize_tensor_range(test_labels)



tensor(0.) tensor(5.)
tensor(0.) tensor(5.)
tensor(0.) tensor(5.)


In [14]:
normalized_labels

tensor([1.0000, 0.7600, 0.7600,  ..., 0.0000, 0.0000, 0.0000])

In [15]:
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], normalized_labels)
validation_dataset = TensorDataset(valid_inputs['input_ids'], valid_inputs['attention_mask'], valid_normalized_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_normalized_labels)


In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [17]:
valid_dataloader = DataLoader(validation_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [18]:
# for batch in train_dataloader:
#     print(batch)
#     break

In [19]:
from transformers import BertTokenizer, BertModel

In [20]:
bert = BertModel.from_pretrained('bert-base-uncased')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [21]:
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class CrossEncoder(nn.Module):

  def __init__(self, bert):
    super().__init__()
    self.bert = bert
    self.dropout = nn.Dropout(0.3)
    self.linear = nn.Linear(768, 1)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    bert_output = outputs.pooler_output
    bert_output = self.dropout(bert_output)
    return self.linear(bert_output)

In [23]:
from tqdm import tqdm
def train(epochs, lr, model, train_dataloader, valid_dataloader, opt_func=torch.optim.Adam):
  history = []
  optimizer = opt_func(model.parameters(), lr)
  for epoch in range(epochs):
    model.train()
    train_losses = []
    valid_losses = []
    for batch in tqdm(train_dataloader):
      input_ids = batch[0].to(device)
      attention_mask = batch[1].to(device)
      labels = batch[2].to(device)

      output = model(input_ids=input_ids, attention_mask=attention_mask)
      output = torch.sigmoid(output).squeeze()
      loss = torch.nn.MSELoss()(output, labels)
      train_losses.append(loss)
      loss.backward() #calculate gradients
      optimizer.step()
      optimizer.zero_grad()
    total_loss = torch.stack(train_losses).mean().item()
    print('epoc_training_loss', total_loss)
    # history.append(total_loss)

    model.eval()
    with torch.no_grad():
      for batch in tqdm(valid_dataloader):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        output = model(input_ids=input_ids, attention_mask=attention_mask)
        output = torch.sigmoid(output).squeeze()
        loss = torch.nn.MSELoss()(output, labels)
        valid_losses.append(loss)
      total_loss = torch.stack(valid_losses).mean().item()
      print('epoc_validation_loss', total_loss)
    # history.append(total_loss)


In [24]:
model = CrossEncoder(bert)
model.to(device)
model.train()

CrossEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [25]:
epochs = 1
lr = 0.00001
history = train(epochs, lr, model, train_dataloader, valid_dataloader)

100%|██████████| 90/90 [01:38<00:00,  1.09s/it]


epoc_training_loss 0.06133834645152092


100%|██████████| 24/24 [00:06<00:00,  3.66it/s]

epoc_validation_loss 0.04232969135046005


In [26]:
epochs = 6
lr = 0.00001
history = train(epochs, lr, model, train_dataloader, valid_dataloader)

100%|██████████| 90/90 [01:38<00:00,  1.10s/it]


epoc_training_loss 0.03187185153365135


100%|██████████| 24/24 [00:06<00:00,  3.71it/s]


epoc_validation_loss 0.03276202082633972


100%|██████████| 90/90 [01:38<00:00,  1.09s/it]


...

epoc_validation_loss 0.027536040171980858


100%|██████████| 90/90 [01:38<00:00,  1.10s/it]


epoc_training_loss 0.01177204493433237


100%|██████████| 24/24 [00:06<00:00,  3.70it/s]

epoc_validation_loss 0.025993099436163902


In [27]:
epochs = 2
lr = 0.00001
history = train(epochs, lr, model, train_dataloader, valid_dataloader)

100%|██████████| 90/90 [01:38<00:00,  1.10s/it]


epoc_training_loss 0.010825000703334808


100%|██████████| 24/24 [00:06<00:00,  3.71it/s]


epoc_validation_loss 0.026259154081344604


100%|██████████| 90/90 [01:38<00:00,  1.10s/it]


epoc_training_loss 0.008825620636343956


100%|██████████| 24/24 [00:06<00:00,  3.70it/s]

epoc_validation_loss 0.02578015625476837
